# [CSC8101] Big Data Analytics - 2022 Spark Coursework

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/NewcastleComputingScience/csc8101-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/parquet/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/delta/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


In [0]:
print_count(trips)

Row count: 2,898,033


In [0]:
# dataset schemas
trips.printSchema()

root
 |-- index: long (nullable = true)
 |-- VendorID: double (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- cab_type: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |--

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3


In [0]:
zone_names.printSchema()

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.483 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    # input: trips dataset
    df = (trips.filter((trips.trip_distance != 0) | ((trips.passenger_count != 0) & (trips.total_amount != 0))))
    return df

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)
#display(trips_11[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(10))

## uncomment only for smaller datasets
display(trips_11)

Row count: 2,890,208


index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,cab_type,lpep_pickup_datetime,lpep_dropoff_datetime,ehail_fee,trip_type
0,1.0,2021-07-01 00:08:51,2021-07-01 00:13:05,1.0,0.8,1.0,N,90,68,1.0,5.0,3.0,0.5,0.0,0.0,0.3,8.8,2.5,Y,null,null,null,null
1,1.0,2021-07-01 00:22:39,2021-07-01 00:25:58,1.0,0.9,1.0,N,113,90,2.0,5.0,3.0,0.5,0.0,0.0,0.3,8.8,2.5,Y,null,null,null,null
2,1.0,2021-07-01 00:48:33,2021-07-01 00:54:58,1.0,2.8,1.0,N,88,232,2.0,10.0,3.0,0.5,0.0,0.0,0.3,13.8,2.5,Y,null,null,null,null
3,1.0,2021-07-01 00:59:44,2021-07-01 01:07:09,1.0,1.4,1.0,N,79,249,1.0,7.0,3.0,0.5,1.5,0.0,0.3,12.3,2.5,Y,null,null,null,null
4,1.0,2021-07-01 00:08:35,2021-07-01 00:16:28,0.0,2.0,1.0,N,142,238,1.0,8.5,3.0,0.5,0.0,0.0,0.3,12.3,2.5,Y,null,null,null,null
5,1.0,2021-07-01 00:10:49,2021-07-01 00:18:42,1.0,1.6,1.0,N,114,90,1.0,7.5,3.0,0.5,1.5,0.0,0.3,12.8,2.5,Y,null,null,null,null
6,1.0,2021-07-01 00:28:56,2021-07-01 00:36:11,1.0,1.8,1.0,N,90,144,1.0,7.5,3.0,0.5,2.0,0.0,0.3,13.3,2.5,Y,null,null,null,null
7,1.0,2021-07-01 00:45:10,2021-07-01 00:52:46,1.0,2.0,1.0,N,114,48,1.0,8.5,3.0,0.5,2.45,0.0,0.3,14.75,2.5,Y,null,null,null,null
8,1.0,2021-07-01 00:55:43,2021-07-01 01:10:56,1.0,5.7,1.0,N,48,152,2.0,18.5,3.0,0.5,0.0,0.0,0.3,22.3,2.5,Y,null,null,null,null
9,1.0,2021-07-01 00:04:24,2021-07-01 00:13:23,1.0,1.8,1.0,N,234,148,1.0,8.5,3.0,0.5,2.45,0.0,0.3,14.75,2.5,Y,null,null,null,null


In [0]:
# Your solution implementation to task 1.2 goes HERE
def t12_remove_outliers(df):
    
    import pandas as pd
    from pyspark.sql.functions import expr, percentile_approx
    from pyspark.sql.functions import abs
    
    #for column "total_amount"....
    
    median_amount = df.agg(percentile_approx("total_amount",0.5).alias('median'))
    df = df.withColumn("median_diff", df["total_amount"] - median_amount.collect()[0][0])
    df = df.withColumn("median_diff_abs", abs(df.median_diff))
    
    df2 = df.agg(percentile_approx("median_diff_abs", 0.5).alias('median'))
    mad = 1.483 * df2.collect()[0][0]
    df = df.withColumn("z_score", df["median_diff_abs"]/mad)
    df = df.filter(df.z_score < 3.5)
    
    #for column "trip_distance"....
    
    median_amount = df.agg(percentile_approx("trip_distance",0.5).alias('median'))
    df = df.withColumn("median_diff1", df["trip_distance"] - median_amount.collect()[0][0])
    df = df.withColumn("median_diff_abs1", abs(df.median_diff1))
    
    df2 = df.agg(percentile_approx("median_diff_abs1", 0.5).alias('median'))
    mad = 1.483 * df2.collect()[0][0]
    df = df.withColumn("z_score1", df["median_diff_abs1"]/mad)
    df = df.filter(df.z_score1 < 3.5)
    return df

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
display(trips_12)

Row count: 2,463,077


index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,cab_type,lpep_pickup_datetime,lpep_dropoff_datetime,ehail_fee,trip_type,median_diff,median_diff_abs,z_score,median_diff1,median_diff_abs1,z_score1
0,1.0,2021-07-01 00:08:51,2021-07-01 00:13:05,1.0,0.8,1.0,N,90,68,1.0,5.0,3.0,0.5,0.0,0.0,0.3,8.8,2.5,Y,null,null,null,null,-6.5,6.5,1.0193040505574806,-0.9299999999999999,0.9299999999999999,0.7838840188806474
1,1.0,2021-07-01 00:22:39,2021-07-01 00:25:58,1.0,0.9,1.0,N,113,90,2.0,5.0,3.0,0.5,0.0,0.0,0.3,8.8,2.5,Y,null,null,null,null,-6.5,6.5,1.0193040505574806,-0.83,0.83,0.6995954146999326
2,1.0,2021-07-01 00:48:33,2021-07-01 00:54:58,1.0,2.8,1.0,N,88,232,2.0,10.0,3.0,0.5,0.0,0.0,0.3,13.8,2.5,Y,null,null,null,null,-1.5,1.5,0.2352240116671109,1.0699999999999998,1.0699999999999998,0.9018880647336479
3,1.0,2021-07-01 00:59:44,2021-07-01 01:07:09,1.0,1.4,1.0,N,79,249,1.0,7.0,3.0,0.5,1.5,0.0,0.3,12.3,2.5,Y,null,null,null,null,-3.0,3.0,0.4704480233342218,-0.33000000000000007,0.33000000000000007,0.2781523937963588
4,1.0,2021-07-01 00:08:35,2021-07-01 00:16:28,0.0,2.0,1.0,N,142,238,1.0,8.5,3.0,0.5,0.0,0.0,0.3,12.3,2.5,Y,null,null,null,null,-3.0,3.0,0.4704480233342218,0.27,0.27,0.2275792312879299
5,1.0,2021-07-01 00:10:49,2021-07-01 00:18:42,1.0,1.6,1.0,N,114,90,1.0,7.5,3.0,0.5,1.5,0.0,0.3,12.8,2.5,Y,null,null,null,null,-2.5,2.5,0.39204001944518485,-0.1299999999999999,0.1299999999999999,0.10957518543492911
6,1.0,2021-07-01 00:28:56,2021-07-01 00:36:11,1.0,1.8,1.0,N,90,144,1.0,7.5,3.0,0.5,2.0,0.0,0.3,13.3,2.5,Y,null,null,null,null,-2.0,2.0,0.3136320155561479,0.07000000000000006,0.07000000000000006,0.05900202292650039
7,1.0,2021-07-01 00:45:10,2021-07-01 00:52:46,1.0,2.0,1.0,N,114,48,1.0,8.5,3.0,0.5,2.45,0.0,0.3,14.75,2.5,Y,null,null,null,null,-0.5500000000000007,0.5500000000000007,0.08624880427794078,0.27,0.27,0.2275792312879299
8,1.0,2021-07-01 00:55:43,2021-07-01 01:10:56,1.0,5.7,1.0,N,48,152,2.0,18.5,3.0,0.5,0.0,0.0,0.3,22.3,2.5,Y,null,null,null,null,7.0,7.0,1.0977120544465175,3.97,3.97,3.3462575859743766
9,1.0,2021-07-01 00:04:24,2021-07-01 00:13:23,1.0,1.8,1.0,N,234,148,1.0,8.5,3.0,0.5,2.45,0.0,0.3,14.75,2.5,Y,null,null,null,null,-0.5500000000000007,0.5500000000000007,0.08624880427794078,0.07000000000000006,0.07000000000000006,0.05900202292650039


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitabilty = total_amount / trip_distance`.

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
# Your solution implementation to task 2.1 goes HERE
from pyspark.sql.functions import col
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    df = df.join(zones_df, df.PULocationID == zones_df.LocationID).select(df["*"], zones_df["zone"])
    df = df.withColumnRenamed("zone","start")
    
    zones_df = zones_df.select("LocationID",col("zone").alias("New_zone"))
    df = df.join(zones_df, df.DOLocationID == zones_df.LocationID).select(df["*"], zones_df["New_zone"])
    df = df.withColumnRenamed("New_zone", "end")
    return df

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
#display(trips_21[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(10))
#display(trips_21)

Row count: 2,463,077


In [0]:
# Your solution implementation to task 2.2 goes HERE
from pyspark.sql.functions import lit,abs
def t22_calc_profit(df):
    # input: output of task 2.1
    df = df.withColumn("unit_profitability", (df.total_amount / df.trip_distance))
    return df

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
#display(trips_22.take(10))

Row count: 2,463,077


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z2 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitabilty)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
## Your solution to task 3.1 goes HERE
def t31_summarise_trips(df):
    # input: output of task 2.2
    df = df.groupBy("PULocationID", "DOLocationID") \
    .agg(pf.mean("unit_profitability").alias('average_unit_profit'),
         pf.count("VendorID").alias('trips_count'),
         pf.sum("passenger_count").alias('total_passangers'))
    return df

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
display(graph.take(10))

Row count: 14,455


PULocationID,DOLocationID,average_unit_profit,trips_count,total_passangers
90,231,7.432778991914051,1094,1576.0
87,33,6.817160729833857,105,145.0
90,142,6.873745233124353,885,1279.0
114,100,7.480600705110231,453,707.0
234,144,9.131779529653814,1466,2187.0
148,262,4.89353554100773,221,356.0
246,249,8.820105729239454,1108,1713.0
170,179,5.138416672570028,78,103.0
48,232,5.478542842122207,306,474.0
142,144,5.799719215582247,324,482.0


In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    return df

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    from pyspark.sql.functions import sum,col,desc
    df_zones = df_zones.groupBy("PULocationID")\
    .agg(sum("trips_count").alias('trips_count'))\
    .sort(desc("trips_count"))
    return df_zones.take(10)

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    from pyspark.sql.functions import sum,col,desc
    df_zones = df_zones.groupBy("PULocationID")\
    .agg(sum("average_unit_profit").alias('average_unit_profit'))\
    .sort(desc("average_unit_profit"))
    return df_zones.take(10)

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    from pyspark.sql.functions import sum,col,desc
    df_zones = df_zones.groupBy("PULocationID")\
    .agg(sum("total_passangers").alias('total_passangers'))\
    .sort(desc("total_passangers"))
    return df_zones.take(10)

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
top10_trips

Out[23]: [Row(PULocationID=237, trips_count=119773),
 Row(PULocationID=236, trips_count=101232),
 Row(PULocationID=186, trips_count=95458),
 Row(PULocationID=161, trips_count=91806),
 Row(PULocationID=170, trips_count=85819),
 Row(PULocationID=162, trips_count=83207),
 Row(PULocationID=142, trips_count=78903),
 Row(PULocationID=141, trips_count=72254),
 Row(PULocationID=48, trips_count=72183),
 Row(PULocationID=239, trips_count=71978)]

In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
top10_profit

Out[24]: [Row(PULocationID=15, average_unit_profit=3736.4352610622004),
 Row(PULocationID=190, average_unit_profit=3170.761284200941),
 Row(PULocationID=10, average_unit_profit=3099.1068599461696),
 Row(PULocationID=265, average_unit_profit=2508.9955424170016),
 Row(PULocationID=241, average_unit_profit=2494.458083171544),
 Row(PULocationID=89, average_unit_profit=1995.0216510262374),
 Row(PULocationID=25, average_unit_profit=1589.605754195491),
 Row(PULocationID=168, average_unit_profit=1386.5277275323278),
 Row(PULocationID=138, average_unit_profit=1337.3670671761902),
 Row(PULocationID=116, average_unit_profit=1193.921477050753)]

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
top10_passenger

Out[25]: [Row(PULocationID=237, total_passangers=169800.0),
 Row(PULocationID=236, total_passangers=143579.0),
 Row(PULocationID=186, total_passangers=136625.0),
 Row(PULocationID=161, total_passangers=134263.0),
 Row(PULocationID=170, total_passangers=123110.0),
 Row(PULocationID=162, total_passangers=118884.0),
 Row(PULocationID=142, total_passangers=114275.0),
 Row(PULocationID=48, total_passangers=106488.0),
 Row(PULocationID=239, total_passangers=105037.0),
 Row(PULocationID=234, total_passangers=104832.0)]

## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# after developing your solution, it may be convenient to combine all your functions in a single cell (at the start or end of the notebook)

# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'
WITH_TASK_12 = True

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

Out[27]: [[Row(PULocationID=237, trips_count=119773),
  Row(PULocationID=236, trips_count=101232),
  Row(PULocationID=186, trips_count=95458),
  Row(PULocationID=161, trips_count=91806),
  Row(PULocationID=170, trips_count=85819),
  Row(PULocationID=162, trips_count=83207),
  Row(PULocationID=142, trips_count=78903),
  Row(PULocationID=141, trips_count=72254),
  Row(PULocationID=48, trips_count=72183),
  Row(PULocationID=239, trips_count=71978)],
 [Row(PULocationID=15, average_unit_profit=3736.4352610622004),
  Row(PULocationID=190, average_unit_profit=3170.761284200941),
  Row(PULocationID=10, average_unit_profit=3099.1068599461696),
  Row(PULocationID=265, average_unit_profit=2508.9955424170016),
  Row(PULocationID=241, average_unit_profit=2494.458083171544),
  Row(PULocationID=89, average_unit_profit=1995.0216510262374),
  Row(PULocationID=25, average_unit_profit=1589.605754195491),
  Row(PULocationID=168, average_unit_profit=1386.5277275323278),
  Row(PULocationID=138, average_unit

_Table 1. Pipeline performance for `parquet` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |2,898,033 |1557166 |41953716|90443069|132396785|
| execution time   (w/o 1.2)  |7.6 |124.5|1150.7|Error:- Threshhole Exceeded| 0.00 |
| execution time              | 13.64 | 245 |2146.2|Error:- Threshhole Exceeded| 0.00 |
| sec / 1M records (w/o 1.2)  | 2.57  | 8.30  |26.90|0.0| 0.0  |
| sec / 1M records            | 4.79  |15.78|51.40| 0.0  | 0.0  |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    | 2,898,033 |1557166|41953716|90443069|132396785|
| execution time   (w/o 1.2)  |5.40|7.02 | 13.5 | 19.40 | 23.45|
| execution time              |10.89 |18.60| 29.20 | 49.70| 62.80 |
| sec / 1M records (w/o 1.2)  |1.9  | 0.63| 0.51 | 0.31 | 0.17|
| sec / 1M records            |3.68| 1.53 | 0.59 | 0.55 | 0.44  |